<a href="https://colab.research.google.com/github/vladmark/Dialect-detection-ML-challenge/blob/main/Dialect%20Detection%20Challenge%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminarii: imports, gdrive setup etc.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
# ! pip install torch
import numpy as np
import pandas as pd
import string
import re
import json
import torch.nn as nn
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import torch.nn.functional as F
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
# nltk.download()



In [ ]:

torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# Partea I: Aducerea de date la un format cu care se poate lucra

##**Procesare de raw data**

Functia de procesat date din fisier

In [ ]:
def get_data(location, type ='samplefile'):
    with open(location, encoding="utf8") as file:
        if(type=='samplefile'):
            data={}
        elif(type=='labelfile'):
            data=torch.empty(1,2)
            i=0
        elif(type == 'labelfile_no_OneHot'):
            data={}
        for line in file:
            id = line[:line.find("	")]
            text = line[line.find("	")+1:]
            if(type=='samplefile'):
                # text = re.sub("(\\d|\\W)+", " ", text)
                # text = text.lower()
                # text = text.split()
                # text = list(text)
                # for word in stopwords.words('romanian'):
                #     if (word in text):
                #         text.remove(word)
                # text = ' '.join(text)
                # id = int(id)
                # text=text.replace("+", " ")
                # text=text.replace("-", " ")
                # text=text.replace(":", " ")
                # text=text.replace(";", " ")
                # text=text.split(" ")
                data[id] = text
            elif (type=='labelfile'):
                if i==0:
                    data[i,:]=torch.Tensor([int(id), int(text)])
                    i = i + 1
                else:
                    data=torch.cat((data, torch.Tensor([[int(id), int(text)]]) ), dim=0)
            elif (type == 'labelfile_no_OneHot'):
                data[id]=int(text)
        if (type == 'labelfile'):
            enc = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
            encoded_labels = enc.fit_transform(data[:, 1].type(torch.IntTensor).reshape(-1,1) )
            dict={}
            for i in range(data.shape[0]):
                dict[int(data[i, 0].item())]=[int(encoded_labels[i,0]), int(encoded_labels[i,1])]
            return(dict)
        elif (type == 'labelfile_no_OneHot'):
            return data
        elif (type == 'samplefile'):
            return data


Functii de manipulat json:

In [ ]:
def write_to_file(data, filename):
    jsonloc = json.dumps(data)
    f = open(filename,"w")
    f.write(jsonloc)
    f.close()

def get_data_from_file(filename):
    f=open(filename,"r")
    data=json.load(f)
    f.close()
    return(data)

In [ ]:
basepath='/content/drive/My Drive/Proiect IA/'

Creeare de jsonuri **daca nu sunt create deja**

In [ ]:
train_data=get_data(basepath+'train_samples.txt')
dict_train_labels=get_data(basepath+'train_labels.txt', type='labelfile_no_OneHot')
write_to_file(train_data, basepath+"dict_train_data.json")
write_to_file(dict_train_labels, basepath+"dict_train_labels.json")

In [ ]:
validation_data = get_data(basepath+'validation_samples.txt')
dict_validation_labels = get_data(basepath+'validation_labels.txt', type='labelfile_no_OneHot')
write_to_file(validation_data, basepath+"dict_validation_data.json")
write_to_file(dict_validation_labels, basepath+"dict_validation_labels.json")

In [ ]:
test_data=get_data(basepath+'test_samples.txt')
write_to_file(test_data, basepath+"dict_test_data.json")

Incarcare din jsonuri **daca exista deja**

In [ ]:
train_data=get_data_from_file(basepath+"dict_train_data.json")
validation_data = get_data_from_file(basepath+"dict_validation_data.json")

In [ ]:
dict_train_labels=get_data_from_file(basepath+"dict_train_labels.json")
dict_validation_labels = get_data_from_file(basepath+"dict_validation_labels.json")

In [ ]:
test_data=get_data_from_file(basepath+"dict_test_data.json")


Procesare labeluri.

In [ ]:
train_labels = torch.LongTensor([dict_train_labels[key] for key in dict_train_labels.keys()])
validation_labels = torch.LongTensor([dict_validation_labels[key] for key in dict_validation_labels.keys()])

In [ ]:
train_textlist=[train_data[key].split() for key in train_data.keys()]
validation_textlist=[validation_data[key].split() for key in validation_data.keys()]
test_textlist=[test_data[key].split() for key in test_data.keys()]

##**Observatii pe date**

###Basics

In [ ]:
print(f'numarul de texte de train: {len(train_textlist)}')
print(f'numarul de texte de validare: {len(validation_textlist)}')
print(f'numarul de texte de test: {len(test_textlist)}')
print(f'exemplu de text: {train_textlist[46]}' )

numarul de texte de train: 7757
numarul de texte de validare: 2656
numarul de texte de test: 2623
exemplu de text: ['|eF!', 'a:Z@', 'pg', '@<wtq', 'c=.HUz', 'fhk#', 'XBvC', 'rl#:', '|@UD', '&e', 'RElTp', 'em@#', 'op!', 'AxZF', 'i&:', 'mk@kchk', 'gkea&hk', 'pyXA', "'Bh", 'hkm}hf@m', 'mghZ', 'AgXwB', '*cp>h', '@:fh']


### Statistici

Numarul de exemple din fiecare clasa pentru setul de train si setul de validare.

In [ ]:
no_labels_train=torch.FloatTensor([torch.sum(train_labels == 1.), torch.sum(train_labels == 0.)])*(1/train_labels.shape[0])
no_labels_validation=torch.FloatTensor([torch.sum(validation_labels == 1.), torch.sum(validation_labels == 0.)])*(1/validation_labels.shape[0])
print(f'distributia claselor pe train: {no_labels_train}')
print(f'distributia claselor pe validare: {no_labels_validation}')

distributia claselor pe train: tensor([0.5068, 0.4932])
distributia claselor pe validare: tensor([0.5102, 0.4898])


Lungimile textelor de train/validare/test; lungimiile medii ale textelor si variantele lungimilor textelor.

In [ ]:
train_text_lengths=torch.FloatTensor([len(train_data[key]) for key in train_data.keys()])
validation_text_lengths=torch.FloatTensor([len(validation_data[key]) for key in validation_data.keys()])
test_text_lengths=torch.FloatTensor([len(test_data[key]) for key in test_data.keys()])
print(f'lungimile textelor de train: {train_text_lengths}')
print(f'lungimile textelor de validare: {validation_text_lengths}')
print(f'lungimile textelor de test: {test_text_lengths}')
print(f'media lungimilor textelor de train: {torch.mean(train_text_lengths)}')
print(f'media lungimilor textelor de validare: {torch.mean(validation_text_lengths)}')
print(f'media lungimilor textelor de test: {torch.mean(test_text_lengths)}')
print(f'deviatia standard a textelor de train: {torch.std(train_text_lengths)}')
print(f'deviatia standard a textelor de validare: {torch.std(validation_text_lengths)}')
print(f'deviatia standard a lungimilor textelor de test: {torch.std(test_text_lengths)}')

lungimile textelor de train: tensor([349.,  90., 169.,  ..., 785.,  88., 207.])
lungimile textelor de validare: tensor([165., 214., 205.,  ..., 165., 305., 180.])
lungimile textelor de test: tensor([124., 273., 233.,  ...,  63., 148., 211.])
media lungimilor textelor de train: 182.95204162597656
media lungimilor textelor de validare: 186.8911895751953
media lungimilor textelor de test: 176.7014923095703
deviatia standard a textelor de train: 192.9701690673828
deviatia standard a textelor de validare: 192.45263671875
deviatia standard a lungimilor textelor de test: 149.76451110839844


Lungimile medii ale cuvintelor per fiecare text din fiecare dataset, apoi media lungimilor medii ale cuvintelor pe fiecare dataset si deviata satandard.

In [ ]:
#average of len of words of each text
average_train_word_lengths_per_text=[np.array(([len(train_textlist[i][j]) for j in range(len(train_textlist[i]))]) ).mean() for i in range(len(train_textlist))]
average_validation_word_lengths_per_text=[np.array(([len(validation_textlist[i][j]) for j in range(len(validation_textlist[i]))]) ).mean() for i in range(len(validation_textlist))]
average_test_word_lengths_per_text=[np.array(([len(test_textlist[i][j]) for j in range(len(test_textlist[i]))]) ).mean() for i in range(len(test_textlist))]

average_train_word_lengths_per_text=torch.FloatTensor(average_train_word_lengths_per_text)
average_validation_word_lengths_per_text=torch.FloatTensor(average_validation_word_lengths_per_text)
average_test_word_lengths_per_text=torch.FloatTensor(average_test_word_lengths_per_text)
print(f'mediile lungimiilor cuvintelor per fiecare text de train: {average_train_word_lengths_per_text}')
print(f'mediile lungimiilor cuvintelor per fiecare text de validare: {average_validation_word_lengths_per_text}')
print(f'mediile lungimiilor cuvintelor per fiecare text de test: {average_test_word_lengths_per_text}')
print(f'media mediilor lungimiilor cuvintelor per text de train: {torch.mean(average_train_word_lengths_per_text)}')
print(f'media mediilor lungimiilor cuvintelor per text de validare: {torch.mean(average_validation_word_lengths_per_text)}')
print(f'media mediilor lungimiilor cuvintelor per text de test: {torch.mean(average_test_word_lengths_per_text)}')
print(f'deviatia standard a mediilor lungimiilor cuvintelor per text de train: {torch.std(average_train_word_lengths_per_text)}')
print(f'deviatia standard a mediilor lungimiilor cuvintelor per text de validare: {torch.std(average_validation_word_lengths_per_text)}')
print(f'deviatia standard a mediilor lungimiilor cuvintelor per text de test: {torch.std(average_test_word_lengths_per_text)}')

mediile lungimiilor cuvintelor per fiecare text de train: tensor([5.0172, 4.0000, 4.6333,  ..., 4.9470, 4.5000, 4.4474])
mediile lungimiilor cuvintelor per fiecare text de validare: tensor([4.3226, 4.7838, 4.1250,  ..., 4.5000, 4.6481, 4.8065])
mediile lungimiilor cuvintelor per fiecare text de test: tensor([4.6364, 3.9636, 4.6829,  ..., 4.7273, 4.2857, 4.1463])
media mediilor lungimiilor cuvintelor per text de train: 4.512233734130859
media mediilor lungimiilor cuvintelor per text de validare: 4.498715400695801
media mediilor lungimiilor cuvintelor per text de test: 4.51282262802124
deviatia standard a mediilor lungimiilor cuvintelor per text de train: 0.3247920274734497
deviatia standard a mediilor lungimiilor cuvintelor per text de validare: 0.32956647872924805
deviatia standard a mediilor lungimiilor cuvintelor per text de test: 0.3234666883945465


### Cuvinte care apar in plus pe testul de validare

Cream trei word counturi, pentru datele de train, validare si test respectiv. 

Vrem sa vedem daca apar cuvinte diferite intre dataseturi, deoarece un anumit cuvant care apare intr-un text ar putea conditiona apartenenta textului la dialect, deci poate fi un feature important.

`train_textlist` si celelalte textlisturi sunt deja splituite in cuvinte, deci ca tokenizer apelez o functie dummy care doar returneaza lista cu cuvintele din document

In [ ]:
def dummy_fun(doc):
  return doc

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
wordcounter_train=CountVectorizer(max_df=1., min_df=0, lowercase=False, 
    tokenizer=dummy_fun, preprocessor=dummy_fun, 
    token_pattern=None, binary=True)
wordcounter_validation=CountVectorizer(max_df=1., min_df=0, lowercase=False, 
    tokenizer=dummy_fun, preprocessor=dummy_fun, 
    token_pattern=None, binary=True)
wordcounter_test=CountVectorizer(max_df=1., min_df=0, lowercase=False, 
    tokenizer=dummy_fun, preprocessor=dummy_fun, 
    token_pattern=None, binary=True)
wordcounter_train=wordcounter_train.fit(train_textlist)
wordcounter_validation=wordcounter_validation.fit(validation_textlist)
wordcounter_test=wordcounter_test.fit(test_textlist)

Cheile dictionarului construit de fiecare wordcounter vor fi cuvintele din fiecare dataset. Construim multimi ca sa facem operatii usoare cu ele.

In [ ]:
train_vocab_set=set([key for key in wordcounter_train.vocabulary_.keys()])
validation_vocab_set=set([key for key in wordcounter_validation.vocabulary_.keys()])
test_vocab_set=set([key for key in wordcounter_test.vocabulary_.keys()])

In [ ]:
print(len(train_vocab_set))

32874


In [ ]:
print(f'numar total de cuvinte din train: {len(train_vocab_set)}')
print(f'numar total de cuvinte din validation: {len(validation_vocab_set)}')
print(f'numar total de cuvinte din test: {len(test_vocab_set)}')
print(f'numar de cuvinte care sunt in validation si nu sunt in train: \
{len(validation_vocab_set - train_vocab_set)}\
; procentual: {len(validation_vocab_set - train_vocab_set)/len(validation_vocab_set)}')
print(f'numar de cuvinte care sunt in test si nu sunt in train: \
{len(test_vocab_set - train_vocab_set)}\
; procentual: {len(test_vocab_set - train_vocab_set)/len(test_vocab_set)}')
print(f'numar de cuvinte care sunt in test si nu sunt nici in validation si nici in train: \
{len(test_vocab_set - (train_vocab_set | validation_vocab_set) )}\
; procentual: {len(test_vocab_set - (train_vocab_set | validation_vocab_set))/len(test_vocab_set)}')

numar total de cuvinte din train: 32874
numar total de cuvinte din validation: 18912
numar total de cuvinte din test: 18104
numar de cuvinte care sunt in validation si nu sunt in train: 5507; procentual: 0.2911907783417936
numar de cuvinte care sunt in test si nu sunt in train: 5158; procentual: 0.284909412284578
numar de cuvinte care sunt in test si nu sunt nici in validation si nici in train: 4457; procentual: 0.24618868758285462


### Cuvinte care apar in datele de antrenament doar pe o eticheta versus cuvinte care apar in validare doar pe eticheta respectiva

In [ ]:
#pastram textele de train si validare care corespund fiecarei etichete
train_label0=[train_textlist[i] for i in range(len(train_textlist)) if train_labels[i]==0]
train_label1=[train_textlist[i] for i in range(len(train_textlist)) if train_labels[i]==1]
validation_label0=[validation_textlist[i] for i in range(len(validation_textlist)) if validation_labels[i]==0]
validation_label1=[validation_textlist[i] for i in range(len(validation_textlist)) if validation_labels[i]==1]
assert len(train_label1)+len(train_label0) == len(train_textlist), "Nu obtin toate textele de train"
assert len(validation_label1)+len(validation_label0) == len(validation_textlist), "Nu obtin toate textele de validare"

In [ ]:
#obtinem vocabularele fiecarei clase din fiecare dataset

wordcounter_train_label0=CountVectorizer(max_df=1., min_df=0, lowercase=False, 
    tokenizer=dummy_fun, preprocessor=dummy_fun, 
    token_pattern=None, binary=True)
wordcounter_train_label1=CountVectorizer(max_df=1., min_df=0, lowercase=False, 
    tokenizer=dummy_fun, preprocessor=dummy_fun, 
    token_pattern=None, binary=True)
wordcounter_val_label0=CountVectorizer(max_df=1., min_df=0, lowercase=False, 
    tokenizer=dummy_fun, preprocessor=dummy_fun, 
    token_pattern=None, binary=True)
wordcounter_val_label1=CountVectorizer(max_df=1., min_df=0, lowercase=False, 
    tokenizer=dummy_fun, preprocessor=dummy_fun, 
    token_pattern=None, binary=True)

wordcounter_train_label0=wordcounter_train_label0.fit(train_label0)
wordcounter_train_label1=wordcounter_train_label1.fit(train_label1)
wordcounter_val_label0=wordcounter_val_label0.fit(validation_label0)
wordcounter_val_label1=wordcounter_val_label1.fit(validation_label1)

In [ ]:
#obtinem seturi de cuvinte din fiecare clasa din fiecare dataset
train_label0_vocab_set=set([key for key in wordcounter_train_label0.vocabulary_.keys()])
train_label1_vocab_set=set([key for key in wordcounter_train_label1.vocabulary_.keys()])
val_label0_vocab_set=set([key for key in wordcounter_val_label0.vocabulary_.keys()])
val_label1_vocab_set=set([key for key in wordcounter_val_label1.vocabulary_.keys()])
assert len(train_label0_vocab_set | train_label1_vocab_set) == len(train_vocab_set),\
       "Cele doua clase din train nu dau tot vocabularul"
assert len(val_label0_vocab_set | val_label1_vocab_set) == len(validation_vocab_set),\
       "Cele doua clase din validare nu dau tot vocabularul"

In [ ]:
#obtinem seturi de cuvinte unice fiecarei clase in interiorul fiecarui dataset
train_label0_minus_label1=train_label0_vocab_set - train_label1_vocab_set
train_label1_minus_label0=train_label1_vocab_set - train_label0_vocab_set
val_label0_minus_label1=val_label0_vocab_set - val_label1_vocab_set
val_label1_minus_label0=val_label1_vocab_set - val_label0_vocab_set

In [ ]:
print(f'numar de cuvinte care apar in datele de train in texte cu eticheta 0 \n si nu apar in niciun text (de train)  \
 cu eticheta 1: {len(train_label0_minus_label1)};\n procentual din toate cuvintele care apar in train cu eticheta 0: \
{len(train_label0_minus_label1)/len(train_label0_vocab_set)}\n')

print(f'numar de cuvinte care apar in datele de train in texte cu eticheta 1 \n si nu apar in niciun text (de train) \
cu eticheta 0: {len(train_label1_minus_label0)}; \n procentual din toate cuvintele care apar in train cu eticheta 0: \
{len(train_label1_minus_label0)/len(train_label1_vocab_set)}\n')

print(f'numar de cuvinte care apar in datele de validare in texte cu eticheta 0 \n si nu apar in niciun text (de validare)\
cu eticheta 1: {len(val_label0_minus_label1)}; \n procentual din toate cuvintele care apar in validare cu eticheta 0: \
{len(val_label0_minus_label1)/len(val_label0_vocab_set)}\n')

print(f'numar de cuvinte care apar in datele de validare in texte cu eticheta 1 \n si nu apar in niciun text (de validare)\
 cu eticheta 0: {len(val_label1_minus_label0)}; \n procentual din toate cuvintele care apar in validare cu eticheta 0: \
{len(val_label1_minus_label0)/len(val_label1_vocab_set)}\n')

numar de cuvinte care apar in datele de train in texte cu eticheta 0 
 si nu apar in niciun text (de train)   cu eticheta 1: 11740;
 procentual din toate cuvintele care apar in train cu eticheta 0: 0.5129325410695561

numar de cuvinte care apar in datele de train in texte cu eticheta 1 
 si nu apar in niciun text (de train) cu eticheta 0: 9986; 
 procentual din toate cuvintele care apar in train cu eticheta 0: 0.47250875366707673

numar de cuvinte care apar in datele de validare in texte cu eticheta 0 
 si nu apar in niciun text (de validare)cu eticheta 1: 6603; 
 procentual din toate cuvintele care apar in validare cu eticheta 0: 0.4932397101665795

numar de cuvinte care apar in datele de validare in texte cu eticheta 1 
 si nu apar in niciun text (de validare) cu eticheta 0: 5525; 
 procentual din toate cuvintele care apar in validare cu eticheta 0: 0.4488585587781298



In [ ]:
print(f'numar de cuvinte care apar in datele de train exclusiv in texte cu eticheta 0 \n\
si apar si in datele de validare exclusiv in texte cu eticheta 0: \
{len(train_label0_minus_label1 & val_label0_minus_label1)}; \n procentual din toate cuvintele de train \
care apar in texte cu eticheta 1: {len(train_label0_minus_label1 & val_label0_minus_label1)/len(train_label0_vocab_set)}; \n\
procentual din toate cuvintele de train care apar exclusiv \nin texte de train cu eticheta 0: \
{len(train_label0_minus_label1 & val_label0_minus_label1)/len(train_label0_minus_label1)};\nprocentual din toate \
cuvintele de validare care apar \nin texte de validare cu eticheta 0: \
{len(train_label0_minus_label1 & val_label0_minus_label1)/len(val_label0_vocab_set)};\nprocentual din toate \
cuvintele de validare care apar exclusiv \nin texte de validare cu eticheta 0: \
{len(train_label0_minus_label1 & val_label0_minus_label1)/len(val_label0_minus_label1)}')

print('\n')

print(f'numar de cuvinte care apar in datele de train exclusiv in texte cu eticheta 1 \n\
si apar si in datele de validare exclusiv in texte cu eticheta 1: \
{len(train_label1_minus_label0 & val_label1_minus_label0)}; \n procentual din toate cuvintele de train \
care apar in texte cu eticheta 1: {len(train_label1_minus_label0 & val_label1_minus_label0)/len(train_label1_vocab_set)}; \n\
procentual din toate cuvintele de train care apar exclusiv \nin texte de train cu eticheta 0: \
{len(train_label1_minus_label0 & val_label1_minus_label0)/len(train_label1_minus_label0)};\nprocentual din toate \
cuvintele de validare care apar \nin texte de validare cu eticheta 0: \
{len(train_label1_minus_label0 & val_label1_minus_label0)/len(val_label1_vocab_set)};\nprocentual din toate \
cuvintele de validare care apar exclusiv \nin texte de validare cu eticheta 0: \
{len(train_label1_minus_label0 & val_label1_minus_label0)/len(val_label1_minus_label0)}')

numar de cuvinte care apar in datele de train exclusiv in texte cu eticheta 0 
si apar si in datele de validare exclusiv in texte cu eticheta 0: 1665; 
 procentual din toate cuvintele de train care apar in texte cu eticheta 1: 0.07274554351625306; 
procentual din toate cuvintele de train care apar exclusiv 
in texte de train cu eticheta 0: 0.1418228279386712;
procentual din toate cuvintele de validare care apar 
in texte de validare cu eticheta 0: 0.1243743930679017;
procentual din toate cuvintele de validare care apar exclusiv 
in texte de validare cu eticheta 0: 0.2521581099500227


numar de cuvinte care apar in datele de train exclusiv in texte cu eticheta 1 
si apar si in datele de validare exclusiv in texte cu eticheta 1: 1209; 
 procentual din toate cuvintele de train care apar in texte cu eticheta 1: 0.05720639727453393; 
procentual din toate cuvintele de train care apar exclusiv 
in texte de train cu eticheta 0: 0.1210694972962147;
procentual din toate cuvintele de validare car

##**Feature extraction din training si validare**. 

Vocabular imbunatatit (nu contine cuvintele comune celor 2 labeluri)

In [ ]:
improved_vocabulary=vocab_label0_not_label1+vocab_label1_not_label0
print(len(improved_vocabulary))
print(len(vocab_label0_not_label1))
print(len(vocab_label1_not_label0))
assert len(improved_vocabulary)==len(vocab_label1_not_label0)+len(vocab_label0_not_label1), print("Ceva n-a mers la extindere")

25901
13830
12071


### **VAR 1: TfIdf**

La validarea sursa o sa convertesc in dense matrix pentru ca e un fisier de 4 ori mai mic si sper sa am destula memorie.

Salvarea matricelor sparse rezultate.

**Daca s-a facut deja, pot incarca din fisier in codul de mai jos**

In [ ]:
#teste (cum arata textlisturile)
print(train_textlist[0])
print(train_textlist[1])
print(validation_textlist[0])
print(validation_textlist[1])

[';%fE', 'mr#&', 'crmx', 'temjc@m', "%'wb:", '}hHAm@@m', 'ykm=aa', 'Eje@', 'Ejh=', 'EcrZk', 's}lZ$', 'rhfh', '}h@kofe@mk', 'RgWE<', '>mfor@m', '@#@', 'm=hkaa', 'TFr>o*', 'h}Ah', 'EHfm}e@mHk', 'e#hj@', 'j&}k', 'gAmaH', 'mgmkafe', 'cmT:', 'k>.h', 'XH(q!', '}FW', '@*oDgB', '#Sx.W', 'hZ', 'jh=', 'chrZ', '}k#h', 'svcNt', 'ejmc@m', 'gYAmZ', 'efke@m', 'h}Ah', 'g@@m', '>m&', '}%k', 'tr(:', ';wxq', 'Ere', 'E*}ga', 'hgZ', 'h$mhr@m', 'tkafe@m', 't@A', '%#sE', '=hkaa@m', 'm*gH', 'E@he=@m', 'wk}hX', 'Ejhr=@m', 'Ejhr=@m', 'h@mg:@']
['sAFW', 'K#xk}t', 'fH@ae', 'm&Xd', '>h&', '@#', 'l@Rd}a', '@Hc', 'liT', 'ehAr@m', 'Xgmz', '!}a', '}eAr@m', 'Be', 'g@@m', 'efH', 'RB(D', 'Ehk&']
['E*Zjv', 'mj=', 'Ee($', 'A@mpe', 'hZ', '.ycn;', '*gjlU', 'H&Zn;', '}kh@Hm}', 'jgA', '}e', 'mj=', 'E@hkx', 'EraZ', '}e', 'mj=', 'hB#*;', 'Eehjck', 'hHrA@m', 'be|KB', "*l'qy", 'e@mA@m', '*|XE', 'Ae', 'mj=', 'H!d', 'h*jx', 'Eejcae', 'mHkrk:', ">c'WY", 'A@mpe']
["Xea'", '}hrAH@m', 'mnSb(', 'Bg:z', 'HA&@@', 'Erkp@m', 'g*g', '}k@pmkae

In [ ]:
def dummy_fun(doc):
  return doc

In [ ]:
#use_idf=false, binary=True, norm=False pentru a obtine un vector binar
#old one: get only whether word is present
# cv=TfidfVectorizer(
#     lowercase=False,
#     token_pattern='[^\s]+')
cv=TfidfVectorizer(
    max_df=0.8, min_df=0, lowercase=False,
    token_pattern='[^\s]+', analyzer='word')
# cv=TfidfVectorizer(
#     max_df=0.8, min_df=0, lowercase=False,
#     token_pattern='[^\s]+', analyzer='char_wb', ngram_range=(1,4))

# cv=TfidfVectorizer(analyzer='char_wb', binary=True, decode_error='replace',
#          encoding='utf-8', input='content',
#         max_df=0.7, min_df=1,
#         ngram_range=(10,12), norm='l2',
#         strip_accents='ascii', sublinear_tf=False,
#                    use_idf=True)
# from sklearn.feature_extraction.text import CountVectorizer
# cv=CountVectorizer(max_df=1., min_df=0, lowercase=False, 
#     tokenizer=dummy_fun, preprocessor=dummy_fun, 
#     token_pattern=None, binary=True, vocabulary=improved_vocabulary)
# word_count_train=cv.fit_transform([train_data[key] for key in train_data.keys()])
cv.fit([train_data[key] for key in train_data.keys()])
word_count_train=cv.transform([train_data[key] for key in train_data.keys()])
word_count_validation=cv.transform([validation_data[key] for key in validation_data.keys()])
# print(type(word_count_train_ngrams))
# print(type(word_count_validation_ngrams))

In [ ]:
# print([train_data[key] for key in train_data.keys()])
print(word_count_train_ngrams.shape)
print(word_count_validation_ngrams.shape)
# print(word_count_train[:3])

(7757, 36871)
(2656, 36871)


In [ ]:
# print([train_data[key] for key in train_data.keys()])
print(word_count_train.shape)
print(word_count_validation.shape)
# print(word_count_train[:3])

(7757, 32874)
(2656, 32874)


In [ ]:
print(word_count_train)

In [ ]:
import scipy
print(word_count_train[:50, :10000].sum(axis=1))

In [ ]:
import scipy
scipy.sparse.save_npz(basepath+'word_count_train', word_count_train, compressed=True)
scipy.sparse.save_npz(basepath+'word_count_validation', word_count_validation, compressed=True)


#### Loading of wordcounts and definition of batch-getting function

Incarc word_counturile **daca exista deja**

In [ ]:
import scipy
word_count_train=scipy.sparse.load_npz(basepath+'word_count_train.npz')
word_count_validation=scipy.sparse.load_npz(basepath+'word_count_validation.npz')
# word_count_target=scipy.sparse.load_npz(basepath+'word_count_target.npz')

In [ ]:
print(word_count_train.shape)

(7757, 25901)


In [ ]:
def binarize(features):
  #supposed to be a non-sparse matrix
  features=torch.Tensor(features)
  if (features.shape[0]==1):
    features=features.squeeze()
    features=[features>0]
    features=features[0]
    features=torch.Tensor([int(i) for i in features])
    features=features.reshape(1, features.shape[0])
  else:
    for i in range(features.shape[0]):
      processing=[features[i,:]>0]
      processing=processing[0]
      processing=torch.Tensor([int(i) for i in processing])
      features[i, :]=processing
  return(features)

Functie care-mi da un batch **pentru TfIdf**. Inputul e asteptat ca sparse matrix.

In [ ]:
def get_batch_tfidf(train_features, train_labels, batch_size, batch_index):
    batch_features=torch.Tensor( train_features[batch_index].todense()).to(device)
    batch_labels=[train_labels[batch_index]]
    for i in range(1,min(batch_index+batch_size, train_features.shape[0])):
        percent=torch.Tensor(train_features[i].todense())
        batch_features=torch.cat((batch_features, percent), dim=0)
        #la fel cu reshape-ul si aici, pentru ca trebuie ca tensorii sa aiba aceeasi dimensiune pentru cat, dar train_labels[i] are o singura dimensiune
        # batch_labels=torch.cat((batch_labels, train_labels[i]))
        batch_labels.append(train_labels[i])
    # print(f'The batch features have shape: {batch_features.shape}')
    return(batch_features, torch.LongTensor(batch_labels).to(device))



### **VAR 2: Word2Vec**

In [ ]:
#average of len of words of each text
print([np.array(([len(train_textlist[i][j]) for j in range(len(train_textlist[i]))]) ).mean() for i in range(len(train_textlist))])

[5.051724137931035, 4.111111111111111, 4.7, 4.3478260869565215, 4.226415094339623, 4.478260869565218, 4.157894736842105, 4.469387755102041, 4.421052631578948, 4.230769230769231, 5.05, 4.543478260869565, 4.666666666666667, 4.7272727272727275, 5.896551724137931, 5.25, 4.5, 4.75, 4.521739130434782, 4.566666666666666, 5.052631578947368, 4.194444444444445, 4.357142857142857, 5.2, 4.571428571428571, 4.981481481481482, 4.56, 4.631578947368421, 4.705882352941177, 5.2, 4.840909090909091, 4.428571428571429, 4.6923076923076925, 4.365384615384615, 4.4, 4.9523809523809526, 4.380952380952381, 4.782608695652174, 4.4, 4.333333333333333, 5.25, 5.7, 4.391304347826087, 3.984375, 4.142857142857143, 4.458333333333333, 4.458333333333333, 4.350877192982456, 4.536332179930795, 4.575757575757576, 4.222222222222222, 4.526315789473684, 4.333333333333333, 4.454545454545454, 4.896551724137931, 3.909090909090909, 4.6891891891891895, 4.954545454545454, 5.0, 4.333333333333333, 4.428571428571429, 4.864864864864865, 4.

In [ ]:
old_traindata=get_data(location=basepath+'old/train_samples.txt')
old_train_textlist=[old_traindata[key] for key in old_traindata.keys()]

Parametrii word2vec: size=1000 e dimensiunea outputului, rezonabil avand in vedere ca avem sute de mii de cuvinte. Cuvintele care apar de mai putine ori decat `min_count` sunt ignorate, dar tocmai acelea ne intereseaza.

In [ ]:
vw_size=5000

In [ ]:
print(train_textlist[0])
print(train_textlist[1])

[';%fE', 'mr#&', 'crmx', 'temjc@m', "%'wb:", '}hHAm@@m', 'ykm=aa', 'Eje@', 'Ejh=', 'EcrZk', 's}lZ$', 'rhfh', '}h@kofe@mk', 'RgWE<', '>mfor@m', '@#@', 'm=hkaa', 'TFr>o*', 'h}Ah', 'EHfm}e@mHk', 'e#hj@', 'j&}k', 'gAmaH', 'mgmkafe', 'cmT:', 'k>.h', 'XH(q!', '}FW', '@*oDgB', '#Sx.W', 'hZ', 'jh=', 'chrZ', '}k#h', 'svcNt', 'ejmc@m', 'gYAmZ', 'efke@m', 'h}Ah', 'g@@m', '>m&', '}%k', 'tr(:', ';wxq', 'Ere', 'E*}ga', 'hgZ', 'h$mhr@m', 'tkafe@m', 't@A', '%#sE', '=hkaa@m', 'm*gH', 'E@he=@m', 'wk}hX', 'Ejhr=@m', 'Ejhr=@m', 'h@mg:@']
['sAFW', 'K#xk}t', 'fH@ae', 'm&Xd', '>h&', '@#', 'l@Rd}a', '@Hc', 'liT', 'ehAr@m', 'Xgmz', '!}a', '}eAr@m', 'Be', 'g@@m', 'efH', 'RB(D', 'Ehk&']


In [ ]:
w2v = Word2Vec(sentences=train_textlist, size=vw_size, min_count=1)

In [ ]:
w2v.save(basepath+"word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Functie care da dataset **pentru Word2Vec**. Inputul e chiar dictionarul de texte si word2vec-ul care le va converti. Ca feature al unui text vom lua suma tuturor vectorilor pentru a avea dimensiune constanta

In [ ]:
def get_word2vec_features(w2vmodel, vw_size: int, data: dict):
    """
    data e un dictionar cu int-uri id-uri de texte si texte, impartite in cuvinte
    labels e un dictionar cu int-uri id-uri de texte si int-uri 0 sau 1 (romaneasca sau moldoveneasca)
    """
    #!ASSUMES DATA IS NOT YET SPLIT
    big_list=[data[key].split(" ") for key in data.keys()]
    features=torch.zeros(len(big_list), vw_size)
    print(f'features shape: {features.shape}')
    for txt_idx in range(len(big_list)):
      for word in big_list[txt_idx]:
        if word in w2vmodel.wv: #daca cuvantul e in vocabularul creeat deja
          word_enc=torch.tensor(w2vmodel[word])
          # print(f'{word} encoded as {word_enc}')
          features[txt_idx]+=word_enc
      features[txt_idx]=(1/len(big_list[txt_idx]))*features[txt_idx]
    return features

In [ ]:
#a test that labels aren't messed up; WILL NOT WORK IF dict_train_labels
read_keys=[key for key in train_data.keys()]
label_keys=[key for key in dict_train_labels.keys()]
for i in range(len(read_keys)):
  print(f'{read_keys[i]}  {label_keys[i]}')

In [ ]:
train_features=get_word2vec_features(w2v, vw_size, train_data)

features shape: torch.Size([7757, 5000])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [ ]:
torch.save(train_features, basepath+'w2v_train_features.pt')

In [ ]:
validation_features = get_word2vec_features(w2v, vw_size, validation_data)

features shape: torch.Size([2656, 5000])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [ ]:
torch.save(validation_features, basepath+'w2v_validation_features.pt')

In [ ]:
print(train_features)
print(validation_features)

tensor([[-0.0374,  0.0446, -0.0241,  ..., -0.0049, -0.0145, -0.0163],
        [-0.0450,  0.0536, -0.0290,  ..., -0.0059, -0.0174, -0.0196],
        [-0.0528,  0.0633, -0.0342,  ..., -0.0070, -0.0207, -0.0231],
        ...,
        [-0.0533,  0.0641, -0.0345,  ..., -0.0071, -0.0209, -0.0233],
        [-0.0222,  0.0266, -0.0144,  ..., -0.0029, -0.0087, -0.0097],
        [-0.0445,  0.0533, -0.0288,  ..., -0.0058, -0.0174, -0.0194]])
tensor([[-0.0703,  0.0844, -0.0455,  ..., -0.0094, -0.0275, -0.0307],
        [-0.0534,  0.0640, -0.0346,  ..., -0.0071, -0.0208, -0.0233],
        [-0.0609,  0.0731, -0.0394,  ..., -0.0081, -0.0238, -0.0265],
        ...,
        [-0.0323,  0.0386, -0.0209,  ..., -0.0042, -0.0126, -0.0141],
        [-0.0665,  0.0797, -0.0430,  ..., -0.0088, -0.0260, -0.0290],
        [-0.0548,  0.0657, -0.0355,  ..., -0.0073, -0.0215, -0.0239]])


In [ ]:
# NO LONGER NEEDED
# validation_target_features=get_word2vec_features(w2v, vw_size, validation_target_data)

features shape: torch.Size([215, 1000])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [ ]:
# NO LONGER NEEDED
# torch.save(validation_target_features, basepath+'w2v_validation_target_features.pt')

Incarca featururile **daca au fost salvate deja** 

In [ ]:
train_features=torch.load(basepath+'w2v_train_features.pt')
validation_features=torch.load(basepath+'w2v_validation_features.pt')
# validation_target_features=torch.load(basepath+'w2v_validation_target_features.pt')

In [ ]:
print(validation_features[:30, 1:10])

Creeaza dataseturi si data loadere

In [ ]:
def get_dataloader(data, labels, batch_size, shuffle=False):
  from torch.utils.data import TensorDataset
  from torch.utils.data import DataLoader
  dataset = TensorDataset(data, labels)
  data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=False)
  return data_loader

In [ ]:
batch_size=800

In [ ]:
train_dataloader=get_dataloader(train_features, train_labels, batch_size, False)
validation_dataloader=get_dataloader(validation_features, validation_labels, batch_size, False)

#NOT NEEDED
# validation_target_dataloader=get_dataloader(validation_target_features, validation_target_labels, batch_size, False)

#Partea II: Configurarea retelelor

## **Clase de retele neuronalle folositle pentru rezolvarea task-ului**

In [ ]:
class ClassifierNN(nn.Module):
  def __init__(self, in_size: int, out_size: int, interm_size: int, no_interm_layers: int):
    super().__init__()
    #apeleaza constructorul clasei mostenite
    self._no_interm_layers=no_interm_layers
    self._layers = [nn.Linear(in_size, interm_size)]
    for i in range(no_interm_layers - 1) :
      self._layers.append(nn.Linear(interm_size, interm_size))
    self._layers.append(nn.Linear(interm_size, out_size))
    self._layers=nn.ModuleList(self._layers)
  def __call__(self, x: torch.Tensor) -> torch.Tensor:
    out = torch.relu(self._layers[0](x))
    for i in range(1, self._no_interm_layers - 1) :
      out = torch.relu(self._layers[i](out))
    out =self._layers[-1](out)
    out=F.log_softmax(out, dim=1)
    return out

In [ ]:
class DescendingClassifier(nn.Module):
  def __init__(self, in_size):
    super().__init__()
    self.f1=nn.Linear(in_size, 1000)
    self.f2=nn.Linear(1000, 100)
    self.f3=nn.Linear(100, 2)
  def __call__(self, x: torch.Tensor) -> torch.Tensor:
    out=torch.relu(self.f1(x))
    out=torch.relu(self.f2(out))
    out=self.f3(out)
    out=torch.log_softmax(out, dim=1)
    return out

##**Functii de training de model:**

### **TfIdf trainer**

In [ ]:
#NOT USED
def lossfun(output, labels):
  #NOT IN USE
  loss=torch.mean(torch.abs(torch.argmax(output, dim=1)-labels).type(torch.FloatTensor))
  return(loss)

In [ ]:
def train_model_tfidf(model, optim, features, true_labels, batch_size, no_epochs, treshold=0.3):
    for e in range(no_epochs):
        for i in range(features.shape[0] // batch_size) :
            optim.zero_grad()
            batch, batch_labels=get_batch_tfidf(features, true_labels, batch_size, i)
            predicted=model(batch)
            loss=torch.nn.functional.nll_loss(predicted, batch_labels)
            loss.backward()
            with torch.no_grad():
                optim.step()
                print(f'epoch {e} has train loss: {loss} on batch {i} (batch_size is {batch_size}')
                # print(f'At epoch {e} model predicts:{predicted[:5]}')
                # print(f'Real lables of batch are: {batch_labels[:5]}')
                from sklearn.metrics import confusion_matrix
                print(f'At epoch {e} on batch {i} confusion matrix is {confusion_matrix(batch_labels, torch.argmax(predicted, dim=1))}')
            if (loss<treshold):
              break
    return(model)

### **Word2Vec trainer**:

In [ ]:
def train_model_word2vec(model, optim, train_loader, batch_size, no_epochs, treshold=0.05):
    for e in range(no_epochs):
        for batch_idx, (batch, batch_labels) in enumerate(train_loader):
            optim.zero_grad()
            predicted=model(batch)
            loss=torch.nn.functional.cross_entropy(predicted, batch_labels)
            loss.backward()
            with torch.no_grad():
                optim.step()
                # print(f'batch is {batch[:10]}')
                # print(f'batch labels are {batch_labels[:10]}')
                print(f'epoch {e} has train loss: {loss} on batch {batch_idx} (batch_size is {batch_size})')
                print(f'At epoch {e} model predicts:{predicted[:5]}')
                print(f'Real lables of batch are: {batch_labels[:5]}')
                print('\n\n\n')
            if (loss<treshold):
              break
    return(model)

def test_model_word2vec(model, test_loader, batch_size):
  total_loss=0
  num_batches=0
  for batch_idx, (batch, batch_labels) in enumerate(test_loader):
    with torch.no_grad():
      predicted=model(batch)
      loss=torch.nn.functional.nll_loss(predicted, batch_labels)
      num_batches+=1
      total_loss+=loss
      print(f'We have test loss: {loss} on batch {batch_idx} (batch_size is {batch_size})')
  total_loss/=num_batches
  return total_loss

##**Training**

Initializare trainig:

In [ ]:
# model=ClassifierNN(word_count_train.shape[1], 2, 15, 2).to(device) #(i, o, interm_size, number of interm layers)
# model=DescendingClassifier(in_size=word_count_train.shape[1]).to(device)
# batch_size=600

model=ClassifierNN(in_size=word_count_train.shape[1], out_size=2, interm_size=10000, no_interm_layers=2).to(device)
# model=DescendingClassifier(in_size=word_count_train.shape[1]).to(device)
# model=ClassifierNN(in_size=train_features.shape[1], out_size=2, interm_size=1000, no_interm_layers=2).to(device)
# model=DescendingClassifier(in_size=train_features.shape[1]).to(device)
optim=torch.optim.Adam(model.parameters(), lr=0.01)

model=model.to(device)

In [ ]:
# print(train_features.shape[1])
print(word_count_train.shape[1])

32874


In [ ]:
no_epochs=1
treshold=0.2
batch_size=800

Training:

In [ ]:
# model=train_model_word2vec(model, optim, train_dataloader, batch_size, no_epochs, treshold)
no_epochs=1
treshold=0.2
batch_size=800
model=train_model_tfidf(model, optim, word_count_train, train_labels, batch_size, no_epochs, treshold)

epoch 0 has train loss: 0.0038216998800635338 on batch 0 (batch_size is 800
At epoch 0 on batch 0 confusion matrix is [[417   0]
 [  0 383]]


In [ ]:
for param in model.parameters():
  print(param)

In [ ]:
batch, _=get_batch_tfidf(word_count_train, train_labels, 50, 0)
print(model(batch))

tensor([[-1.8359, -0.1737],
        [-1.4613, -0.2639],
        [-1.8600, -0.1692],
        [-1.9844, -0.1479],
        [-0.0878, -2.4766],
        [-1.7329, -0.1945],
        [-0.0926, -2.4253],
        [-0.0580, -2.8767],
        [-0.2125, -1.6531],
        [-0.2117, -1.6565],
        [-1.5653, -0.2345],
        [-0.1016, -2.3373],
        [-1.5445, -0.2400],
        [-1.9191, -0.1587],
        [-0.0364, -3.3308],
        [-1.3442, -0.3021],
        [-2.3012, -0.1055],
        [-1.5691, -0.2335],
        [-1.4266, -0.2746],
        [-2.0704, -0.1348],
        [-1.4314, -0.2731],
        [-0.0993, -2.3590],
        [-0.1492, -1.9764],
        [-0.2864, -1.3900],
        [-1.8057, -0.1795],
        [-0.0367, -3.3230],
        [-0.2428, -1.5345],
        [-0.1803, -1.8021],
        [-0.1676, -1.8689],
        [-0.1175, -2.1998],
        [-0.0445, -3.1344],
        [-1.3895, -0.2866],
        [-0.1783, -1.8122],
        [-1.9966, -0.1459],
        [-0.1460, -1.9963],
        [-0.0714, -2

Salvare model. **NOT RUN IF TRAIN HASN'T RUN**

In [ ]:
torch.save(model.state_dict(), basepath+'tfidf_8-12ngrams_classnn_bsize800')

Importare model

In [ ]:
model.load_state_dict(torch.load(basepath+'tfidf_8-12ngrams_classnn_bsize800'))

<All keys matched successfully>

### **Test TfIdf words**

Testare pe datele de validare - **TfIdf**:

In [ ]:
# converteste matricea la una densa
word_count_validation_dense=torch.Tensor(word_count_validation.todense())
output_validation=model(torch.Tensor(word_count_validation_dense))
with torch.no_grad():
  loss_validation=torch.abs(torch.argmax(output_validation, dim=1)-validation_labels)
  loss_validation=loss_validation.type(torch.FloatTensor)
print(f'Validation loss calculated with argmax is {torch.mean(loss_validation)}')
from sklearn.metrics import f1_score
print(f'Validation F1 score is {f1_score(validation_labels, torch.argmax(output_validation, dim=1))}')
from sklearn.metrics import confusion_matrix
print(f'Validation confusion matrix is {confusion_matrix(validation_labels, torch.argmax(output_validation, dim=1))}')

Validation loss calculated with argmax is 0.4100150465965271
Validation F1 score is 0.6289608177172061
Validation confusion matrix is [[644 657]
 [432 923]]


### **Test TfIdf n-grams**

In [ ]:
# converteste matricea la una densa
word_count_validation_ngrams_dense=torch.Tensor(word_count_validation_ngrams.todense())
output_validation=model(torch.Tensor(word_count_validation_ngrams_dense))
with torch.no_grad():
  print(output_validation)
  loss_validation=torch.abs(torch.argmax(output_validation, dim=1)-validation_labels)
  # loss_validation=F.nll_loss(output_validation, validation_labels)
  loss_validation=loss_validation.type(torch.FloatTensor)
print(f'Validation loss is {torch.mean(loss_validation)}')
# print(f'Validation loss is {loss_validation}')

tensor([[-0.5236, -0.8974],
        [-0.4261, -1.0586],
        [-2.2695, -0.1091],
        ...,
        [-0.6792, -0.7073],
        [-1.7652, -0.1877],
        [-1.5446, -0.2400]], grad_fn=<LogSoftmaxBackward>)
Validation loss is 0.36031627655029297


In [ ]:
#train test
# converteste matricea la una densa
word_count_train_ngrams_dense=torch.Tensor(word_count_train_ngrams.todense())
output_validation=model(torch.Tensor(word_count_train_ngrams_dense))
with torch.no_grad():
  print(output_validation)
  loss_validation=torch.abs(torch.argmax(output_validation, dim=1)-train_labels)
  # loss_validation=F.nll_loss(output_validation, validation_labels)
  loss_validation=loss_validation.type(torch.FloatTensor)
print(f'Validation loss is {torch.mean(loss_validation)}')
# print(f'Validation loss is {loss_validation}')

tensor([[-3.0492e+00, -4.8557e-02],
        [-5.6231e+00, -3.6201e-03],
        [-5.0480e+00, -6.4426e-03],
        ...,
        [-2.4009e+00, -9.5008e-02],
        [-1.9962e+00, -1.4601e-01],
        [-2.6289e+00, -7.4890e-02]], grad_fn=<LogSoftmaxBackward>)
Validation loss is 0.3654763400554657


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(validation_labels, torch.argmax(output_validation, dim=1))

array([[864, 437],
       [706, 649]])

### **Test word2vec**:

In [ ]:
total_loss=test_model_word2vec(model, validation_dataloader, batch_size)
print(f'total test loss is {total_loss}')

We have test loss: 0.6807268261909485 on batch 0 (batch_size is 800)
We have test loss: 0.6872389912605286 on batch 1 (batch_size is 800)
We have test loss: 0.686125636100769 on batch 2 (batch_size is 800)
We have test loss: 0.6883438229560852 on batch 3 (batch_size is 800)
total test loss is 0.6856088042259216


### Predicting on test and saving to csv:

In [ ]:
test_data=get_data(basepath+'test_samples.txt')
test_ids=[key for key in test_data.keys()]
write_to_file(test_data, basepath+"dict_test_data.json")
word_count_test=torch.Tensor(cv.transform([test_data[key] for key in test_data.keys()]).todense())
pred_test=model(word_count_test)
test_dict={}

In [ ]:
print(pred_test)
pred_test_args=torch.argmax(pred_test, axis=1)
print(pred_test_args)

tensor([[-1.4314, -0.2731],
        [-0.0135, -4.3101],
        [-1.4291, -0.2738],
        ...,
        [-1.3479, -0.3008],
        [-2.0166, -0.1428],
        [-0.8530, -0.5554]], grad_fn=<LogSoftmaxBackward>)
tensor([1, 0, 1,  ..., 1, 1, 1])


In [ ]:
for i in range(len(test_data.keys())):
  test_dict[test_ids[i]]=int(torch.argmax(pred_test[i]))
print(test_dict)

{'110499': 1, '101319': 0, '108883': 1, '100925': 0, '110852': 0, '109538': 1, '108874': 0, '100082': 1, '110470': 0, '109725': 0, '103874': 1, '108214': 1, '102266': 1, '112940': 1, '102318': 1, '102162': 1, '106268': 1, '101888': 0, '108397': 1, '100901': 1, '106980': 1, '106283': 0, '110891': 0, '103818': 0, '111938': 1, '106200': 0, '109116': 0, '107311': 1, '102213': 1, '112378': 1, '110819': 1, '112456': 1, '110846': 1, '104540': 0, '110136': 1, '110718': 0, '103530': 1, '101797': 0, '110143': 0, '102987': 1, '108970': 0, '102506': 1, '109392': 1, '100695': 0, '103336': 1, '110388': 1, '101995': 0, '109812': 0, '107717': 0, '106342': 0, '112201': 0, '104207': 1, '105057': 0, '100110': 1, '103594': 1, '110747': 0, '101456': 1, '108880': 0, '103008': 1, '112871': 1, '101314': 1, '110408': 1, '110125': 1, '106336': 0, '104844': 1, '101023': 0, '110072': 0, '107032': 1, '105883': 0, '101843': 0, '108219': 0, '110919': 1, '104064': 1, '103193': 0, '100828': 0, '103003': 1, '101164': 1

In [ ]:
arr_test=np.zeros((len(test_ids),  2), dtype=np.int64)
for i in range(len(test_data.keys())):
  arr_test[i, 0]=np.array(test_ids[i], dtype=np.int64)
  arr_test[i, 1]=np.array(pred_test_args[i], dtype=np.int64)
print(arr_test)

[[110499      1]
 [101319      0]
 [108883      1]
 ...
 [103098      1]
 [107046      1]
 [108263      1]]


In [ ]:
np.savetxt("predictii.csv", arr_test, delimiter=",", header='id,label')

# Ahitectura: Recurrent Neural Network

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size: int, embedding_size: int, 
                 rnn_size: int):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.rnn_size = rnn_size
        
        self.embedding=nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size)
        self.rnn_cell = nn.GRUCell(input_size = embedding_size,
                                   hidden_size = rnn_size)
        self.logits = nn.Linear(in_features=rnn_size, out_features=vocab_size)
        self.softmax = nn.Softmax(dim = 2)
        self.loss = nn.CrossEntropyLoss()
    
    def get_loss(self, logits: torch.FloatTensor, y: torch.FloatTensor):
        return self.loss(logits.view(-1,logits.shape[-1]), y.squeeze())
        
    
    def get_logits(self, hidden_states: torch.FloatTensor):
        return self.logits(hidden_states)
        
    def forward(self, text_vectors_raw: torch.LongTensor, 
                hidden_start: torch.FloatTensor = None) -> torch.FloatTensor:
        #batch_size x trim_len x embedding_dim
        text_vectors=self.embedding(text_vectors_raw)
        # print(f'embedded shape: {text_vectors.shape}')
        #compute hidden states and logits for each time step
        hidden_states_list = []
        prev_hidden = hidden_start
        # print(f'previous hidden shape {prev_hidden.shape}')
        for t in range(text_vectors.shape[0]):
            hidden_state = self.rnn_cell(text_vectors[t,:], prev_hidden) #the t-th word of processed text i.e. t-th word vector
            hidden_states_list.append(hidden_state)
            prev_hidden = hidden_state
        
        #one element of hidden_states_list has shape batch_size x max_len x embedding_dim
        hidden_states = torch.stack(hidden_states_list, dim=1)
        # print(f'final hidden states shape: {hidden_states.shape} \n\n')
        return hidden_states

In [ ]:
rnn_size = 50
vocab_size = len(cv.vocabulary_)
embedding_size = 100
max_grad_norm = 5.0

In [ ]:
def make_text_features(text, word_count_model):
    """
    functia ia cuvintele textului primit si le trece pe fiecare prin wordcounter
    apoi creeaza un tensor de dimensiune [nr_cuvinte_din_text x 1]
    label-urile returnate sunt un tensor de dimensiune [nr_cuvinte_din_text x nr_cuvinte_din_vocabular]
    tensorul de returnate contine pe fiecare linie un vector de dimensiunea vocabularului cu 1 in pozitia cuvantului si 0 in rest
    """
    text_words_short=torch.LongTensor( len(text), 1)
    text_actual_words=torch.Tensor(len(text), len(word_count_model.vocabulary_))
    for word_no, word in enumerate(text):
      wcount_emb=torch.tensor(word_count_model.transform([word]).todense())
      text_words_short[word_no]=torch.argmax(wcount_emb).type(torch.LongTensor)
      # text_actual_words[word_no]=torch.tensor(word_count_model.transform([word]).todense())
    return text_words_short

In [ ]:
def train_rnn_epoch(model, optimizer, textlist, word_count_model, max_grad_norm, epoch_num: int) -> float:
        model.train()
        epoch_loss = 0.0
        for text_idx in range(len(textlist)):
            hidden_start = torch.zeros(1, model.rnn_size).to(device)
            text_words_indexes = make_text_features(textlist[text_idx], word_count_model)
            optimizer.zero_grad()
            hidden_states = model(text_words_indexes, hidden_start)
            #rezultatele pentru fiecare bucata de text din batch
            logits = model.get_logits(hidden_states)
            text_loss = model.get_loss(logits, text_words_indexes)
            text_loss += text_loss.item()
                       
            text_loss.backward()
            
            # reducem gradientii la o constanta daca sunt prea mari
            torch.nn.utils.clip_grad_norm_(list(model.parameters()), 
                                           max_grad_norm)
            
            optimizer.step()
            

            if text_idx % 30 == 0:
                print(f'epoch {epoch_num}, text {text_idx} has batch loss = {text_loss.item()}')
        epoch_loss /= text_idx
        
        return epoch_loss


In [ ]:
a=make_text_features(train_textlist[0], cv)
print(a.shape)

torch.Size([58, 1])


In [ ]:
network = RNN(vocab_size=vocab_size,embedding_size=embedding_size, rnn_size=rnn_size)
network = network.to(device)
optimizer = torch.optim.Adam(params = network.parameters(), lr=0.001)

In [ ]:
no_epochs=30
for e in range(1, no_epochs):
  train_rnn_epoch(model=network, optimizer=optimizer, textlist=train_textlist, word_count_model=cv, max_grad_norm=max_grad_norm, epoch_num=e)

epoch 1, text 0 has batch loss = 20.89650535583496
epoch 1, text 100 has batch loss = 20.53720474243164
epoch 1, text 200 has batch loss = 20.449190139770508
epoch 1, text 300 has batch loss = 17.05722427368164
epoch 1, text 400 has batch loss = 20.598848342895508
epoch 1, text 500 has batch loss = 17.35858154296875
epoch 1, text 600 has batch loss = 17.56346321105957
epoch 1, text 700 has batch loss = 16.88450813293457
epoch 1, text 800 has batch loss = 18.41565704345703
epoch 1, text 900 has batch loss = 16.94827651977539
epoch 1, text 1000 has batch loss = 15.07629108428955
epoch 1, text 1100 has batch loss = 15.285298347473145
epoch 1, text 1200 has batch loss = 14.341145515441895
epoch 1, text 1300 has batch loss = 12.853402137756348
epoch 1, text 1400 has batch loss = 15.776110649108887
epoch 1, text 1500 has batch loss = 14.27728271484375
epoch 1, text 1600 has batch loss = 13.286785125732422
epoch 1, text 1700 has batch loss = 13.100395202636719
epoch 1, text 1800 has batch los

KeyboardInterrupt: ignored

In [ ]:
torch.save(network.state_dict(), basepath+'rnn_size50_att2')

In [ ]:
network.load_state_dict(torch.load(basepath+'rnn_size50_att1'))

<All keys matched successfully>

Getting an additional layer on top of it

In [ ]:
class SimpleNN(nn.Module):
  def __init__(self,in_size: int, interm_size: int, out_size: int):
    super().__init__()
    self.lin1=nn.Linear(in_size, interm_size)
    self.lin2=nn.Linear(interm_size, interm_size)
    self.lin3=nn.Linear(interm_size, out_size)
  def forward(self, x):
    out=F.relu(self.lin1(x))
    out=F.relu(self.lin2(out))
    out=self.lin3(out)
    return F.log_softmax(out, dim=1)

In [ ]:
"""functie folosita pana la urma pentru a obtine toate hidden state-urile finale de pe textele de antrenament si validare, pentru ca se poate
antrena reteaua fully-connected pusa peste hidden-state-uri mult mai usor asa"""
def get_rnn_feature(rnn, train_textlist, word_count_model, train_labels, batch_size):
  no_batches=len(train_textlist)//batch_size
  for batch_idx in range(no_batches):
    batch_texts=train_textlist[batch_idx*batch_size: min((batch_idx+1)*(batch_size), len(train_textlist))]
    output_features=torch.Tensor(len(batch_texts), rnn.rnn_size) #vom lua doar ultimul hidden state al retelei pentru fiecare text
    output_labels=train_labels[batch_idx*batch_size: min((batch_idx+1)*batch_size, len(train_labels))]
    for text_id, text in enumerate(batch_texts):
      text_features=make_text_features(text, word_count_model) #returneaza feature-uri de [lg_text x dim_vocab]
      # print(f'text_features after make_text_features {text_features.shape}')
      #trec returnatele prin recurenta
      hidden_start = torch.zeros(1, rnn.rnn_size).to(device)
      recurrent_output=rnn(text_features, hidden_start) #output de dimensiune [lg_text x rnn_size]
      #fac detach ca gradientii sa nu se propage si asupra retelei rnn (deja antrenate)
      recurrent_output.detach_()
      # print(f'recurrent output shape {recurrent_output.shape}')
      #ma intereseaza doar ultimul hidden state (asociat ultimului cuvant din text); pe el il pastrez
      #fac un reshape just in case
      recurrent_output=recurrent_output.view(-1,recurrent_output.shape[-1])
      last_hidden_state=recurrent_output[-1, :]
      # print(f'last_hidden_state shape:{last_hidden_state.shape}')
      output_features[text_id]=last_hidden_state
    print(output_features.shape)
    print(output_labels.shape)
    return output_features, output_labels


In [ ]:
def train_combo(rnn, simple_nn, optimizer_for_simple_nn, rec_train_features, word_count_model, train_labels, batch_size, epoch):
    epoch_loss=0.0
    no_batches=len(train_textlist)//batch_size
    for batch_idx in range(no_batches):
      batch=rec_train_features[batch_idx*batch_size: min((batch_idx+1)*(batch_size), len(train_textlist)), :]
      batch_labels=train_labels[batch_idx*batch_size: min((batch_idx+1)*batch_size, len(train_labels))]
      optimizer_for_simple_nn.zero_grad()
      batch_output=simple_nn(batch)
      loss=F.cross_entropy(batch_output, batch_labels)
      loss.backward()
      optimizer_for_simple_nn.step()
      with torch.no_grad():
          print(f'loss on batch {batch_idx} in epoch {epoch} for the simple nn is: {loss}')

def test_combo(rnn, simple_nn, rec_val_features, word_count_model, validation_labels):
    test_loss=0.0
    with torch.no_grad():
      output=simple_nn(rec_val_features)
      # loss=F.cross_entropy(batch_output, batch_labels)
      print(torch.argmax(output, dim=1))
      print(validation_labels)
      loss = torch.abs(torch.argmax(output, dim=1) - validation_labels).type(torch.FloatTensor)
      print(f'test loss for the simple nn is: {torch.mean(loss)}')
      test_loss+=float(torch.mean(loss))
    return test_loss


In [ ]:
no_epochs_simple_nn=400
batch_size=64

In [ ]:
rec_train_features, _=get_rnn_features(network, train_textlist, cv, train_labels, batch_size=len(train_textlist))

torch.Size([7757, 50])
torch.Size([7757])


In [ ]:
rec_val_features, _=get_rnn_features(network, validation_textlist, cv, validation_labels, batch_size=len(validation_textlist))

torch.Size([2656, 50])
torch.Size([2656])


In [ ]:
torch.save(rec_train_features, basepath+'rec_train_features.pt')
torch.save(rec_train_features, basepath+'rec_validation_features.pt')

In [ ]:
simple_nn=SimpleNN(network.rnn_size, 1000, 2)

In [ ]:
optimizer_for_simple_nn=torch.optim.Adam(simple_nn.parameters(), lr=0.01)

In [ ]:
for e in range(no_epochs_simple_nn):
  train_combo(network, simple_nn, optimizer_for_simple_nn, rec_train_features, cv, train_labels, batch_size, e)

Streaming output truncated to the last 5000 lines.
loss on batch 43 in epoch 52 for the simple nn is: 0.5122823119163513
loss on batch 44 in epoch 52 for the simple nn is: 0.5795986652374268
loss on batch 45 in epoch 52 for the simple nn is: 0.5101519227027893
loss on batch 46 in epoch 52 for the simple nn is: 0.4630879759788513
loss on batch 47 in epoch 52 for the simple nn is: 0.47553592920303345
loss on batch 48 in epoch 52 for the simple nn is: 0.4991553723812103
loss on batch 49 in epoch 52 for the simple nn is: 0.5440242886543274
loss on batch 50 in epoch 52 for the simple nn is: 0.47559598088264465
loss on batch 51 in epoch 52 for the simple nn is: 0.5452596545219421
loss on batch 52 in epoch 52 for the simple nn is: 0.40655383467674255
loss on batch 53 in epoch 52 for the simple nn is: 0.39584875106811523
loss on batch 54 in epoch 52 for the simple nn is: 0.557030439376831
loss on batch 55 in epoch 52 for the simple nn is: 0.49535274505615234
loss on batch 56 in epoch 52 for th

KeyboardInterrupt: ignored

In [ ]:
torch.save(simple_nn.state_dict(), basepath+'simple_nn_epoch3')

In [ ]:
test_combo(network, simple_nn, rec_val_features, cv, validation_labels)

tensor([0, 1, 0,  ..., 0, 0, 0])
tensor([0, 0, 1,  ..., 0, 1, 0])
test loss for the simple nn is: 0.47891566157341003


0.47891566157341003

#MISC: A pre-made model: sklearn SVC

In [ ]:
import sklearn
premodel=sklearn.svm.SVC(degree=2)
premodel.fit(word_count_train, train_labels)
print(premodel.score(word_count_train, train_labels))


0.9984530101843496


In [ ]:
print(premodel.score(word_count_validation, validation_labels))

0.6995481927710844


In [ ]:
import sklearn
premodel_w2v=sklearn.svm.SVC()
premodel_w2v.fit(train_features[:1000], train_labels[:1000])
print(premodel_w2v.score(train_features, train_labels))


0.5645223668944179


In [ ]:
print(premodel_w2v.score(validation_features, validation_labels))

0.545933734939759


In [ ]:
from sklearn.naive_bayes import MultinomialNB
#fit prior is false because classes are uniformly distributed both on train and on validation
premodel_multinomial_NB=MultinomialNB(alpha=0.01, fit_prior=False)
premodel_multinomial_NB.fit(word_count_train, train_labels)
print(premodel_multinomial_NB.score(word_count_train, train_labels))

0.9756349104035065


In [ ]:
print(f'scorul pe datele de validare: {premodel_multinomial_NB.score(word_count_validation, validation_labels)}')
val_prediction=premodel_multinomial_NB.predict(word_count_validation)
print(f'scorul F1 pe datele de validare: {f1_score(validation_labels, val_prediction)}')
print(f'matricea de confuzie pe datele de validare: {confusion_matrix(validation_labels, val_prediction)}')

scorul pe datele de validare: 0.7198795180722891
scorul F1 pe datele de validare: 0.7296511627906976
matricea de confuzie pe datele de validare: [[ 908  393]
 [ 351 1004]]


In [ ]:
print(word_count_train.shape)

(7757, 37101)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
#fit prior is false because classes are uniformly distributed both on train and on validation
premodel_multinomial_NB=MultinomialNB(alpha=0.01, fit_prior=False)
premodel_multinomial_NB.fit(word_count_train_ngrams, train_labels)
print(premodel_multinomial_NB.score(word_count_train_ngrams, train_labels))

0.9739590047698853


In [ ]:
print(premodel_multinomial_NB.score(word_count_validation_ngrams, validation_labels))
val_prediction=premodel_multinomial_NB.predict(word_count_validation_ngrams)
print(f'scorul F1 pe datele de validare: {f1_score(validation_labels, val_prediction)}')
print(f'matricea de confuzie pe datele de validare: {confusion_matrix(validation_labels, val_prediction)}')

0.7127259036144579
scorul F1 pe datele de validare: 0.7242500903505601
matricea de confuzie pe datele de validare: [[ 891  410]
 [ 353 1002]]


In [ ]:
import pickle
pickle.dump(premodel, open(basepath+'svm_model_train_defaultparams', 'wb'))

In [ ]:
premodel = pickle.load(open(basepath+'svm_model_train_defaultparams', 'rb'))

In [ ]:
test_ids=[key for key in test_data.keys()]
word_count_test=torch.Tensor(cv.transform([test_data[key] for key in test_data.keys()]).todense())

In [ ]:
pred_test=premodel_multinomial_NB.predict(word_count_test)
print(pred_test)

[0 0 0 ... 1 1 0]


In [ ]:
arr_test=np.zeros((len(test_ids),  2), dtype=np.int64)
for i in range(len(test_data.keys())):
  arr_test[i, 0]=np.array(test_ids[i], dtype=np.int64)
  arr_test[i, 1]=np.array(pred_test[i], dtype=np.int64)
print(arr_test)

[[110499      0]
 [101319      0]
 [108883      0]
 ...
 [103098      1]
 [107046      1]
 [108263      0]]


In [ ]:
arr_test=np.zeros((len(test_ids),  2), dtype=np.int64)
for i in range(len(test_data.keys())):
  arr_test[i, 0]=np.array(test_ids[i], dtype=np.int64)
  arr_test[i, 1]=np.array(pred_test[i], dtype=np.int64)
print(arr_test)
np.savetxt("predictii.csv", arr_test, delimiter=",", header='id,label')

In [ ]:
from sklearn.naive_bayes import BernoulliNB
premodel_bernoulli=BernoulliNB(alpha=0.001, binarize=0.01, fit_prior=False, class_prior=None)
premodel_bernoulli.fit(word_count_train, train_labels)
print(premodel_bernoulli.score(word_count_train, train_labels))

0.932190279747325


In [ ]:
print(premodel_bernoulli.score(word_count_validation, validation_labels))

0.697289156626506
